# PCA Parameters
<b>n_components</b> : Number of components to keep.<br>
Use Minka's MLE if n_components='mle; & svd_solver=='full' to guess dimension.


<b>copy</b> : Make copy of data, so it is not overwritten with fit. Use fit_transform otherwise.

<b>whiten</b> : If true, components_ vectors are multiplied by sqrt(n_samples) and divided by singular values to ensure uncorrelated outputs with unit component--wise vectors.<br>
<i>Warning:</i> Removes info, but can imporve preditive accuracy by making data respect hard-wired assumptions

<b>svd_solver</b> : Singular Value Decomposition (Lower Dim. Projection Technique)

<i>svd_solver</i>=='auto': Default policy based on X.shape and n_components. Randomized if large and n_components is < 80%. Else Full

<i>svd_solver=='full'</i> : Number of components such that the amount of variance that needs to be explained is greater than % specified.

<i>svd_solver=='randomized'</i> : Halko Method

<b>iterated_power</b> : Number of iterations for power method computed by svd_solver == 'randomized'.

<b>random_state</b> : Random Seed used by randomized.

# PCA Attributes
<b>components_</b> : Principal axes in feature space, represents directions of maximum variance in data.

<b>explained__variance__</b> : Amount of variance explained by each of the selected components.

<b>explained__variance_ratio__</b> : Percent of variance explained by each selected components.

<b>mean_</b> : Estimated number of components. When set to mle or 0 < n_components < 1, number is estimated from input data. Otherwise it equals parameter n_components.

<b>noise_variance</b> : Estimated noise covariance from Bishop PCA.


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

In [3]:
# Interested in Feature space with > 99% Variance
def pca_train_viz(x_train):
    y_train = x_train["TARGET"]
    x_train = x_train.drop(labels="TARGET", axis=1)
    classes = np.sort(np.unique(y_train))
    labels = ["Satisfied", "Unsatisfied"]
    
    #Normalize features to unit norm
    x_train_norm = normalize(x_train, axis=0)
    
    # Run
    pca = PCA(n_components=2)
    x_train_proj = pca.fit_transform(x_train_norm)
    
    # Visuals
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot(1,1,1)
    colors = [(0.0, 0.63, 0.69), 'black']
    markers = ["o", "D"]
    for class_ix, marker, color, label in zip(
        classes, markers, colors, labels):
        ax.scatter(x_train_proj[np.where(y_train == class_ix), 0],
                  x_train_proj[np.where(y_train == class_ix), 1],
                  marker=marker, color=color, edgecolor='whitesmoke',
                  linewidth='1', alpha=0.9, label=label)
        ax.legend(loc='best')
    plt.title("Scatter of Training data Projected on 2 Principal Components")
    plt.xlabel("PCA 1 - Explains %.1f %% of the variance" % (
        pca.explained_variance_ratio_[0] * 100.0))
    plt.ylabel("PCA 2 - Explains %.1f %% of the variance" % (
        pca.explained_variance_ratio_[1] * 100.0))
    plt.show()
    
    # plt.savefig("pca.pdf", format='pdf')
    # plt.savefig("pca.png", format='png')

In [4]:
# Remove feature vector with 1 unique value
# deoesn't help with prediction
def remove_const_feat(X):
    print("Delete Zero Variance Features.\n")
    
    # Get 0 var features by fitting VarThresh to data
    n_feat_orig = X.shape[1]
    selector = VarianceThreshold()
    selector.fit(X)
    
    #Get Zero Variance Indicies
    idx_feat_keep = selector.get_support(indices=True)
    idx_feat_orig = np.arange(X.columns.size)
    idx_feat_delete = np.delete(idx_feat_orig, idx_feat_keep)
    
    # Delete Zero Var Feats
    X = X.drop(labels=X.columns[idx_feat_delete], axis=1)
    
    # Print Actions
    n_feat_delete = idx_feat_delete.size
    print("Deleted %s / %s Features (~ %.1f %%)" % (
        n_feat_delete, n_feat_orig, 
        100.0 * (np.float(n_feat_delete) / n_feat_orig)))
    
    return X

In [5]:
# Removed Identical Deletion
X = pd.read_csv('./Santander/train.csv', index_col=0, sep=',')
X = remove_const_feat(X)
pca_train_viz(X)

Delete Zero Variance Features.



Deleted 34 / 370 Features (~ 9.2 %)
